In [0]:
#This Code predict Label for Roslyn Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019.csv', encoding='latin-1')

y = list(data['Label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:800])
y_train = np.asarray(y_enc[:800])


x_test = np.asarray(x_enc[800:])
y_test = np.asarray(y_enc[800:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [41]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [45]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=60, batch_size=32)
    model.save_weights('./model.h5')

Epoch 1/60
800/800 [==============================] - 11s 13ms/step - loss: 5.1870 - acc: 0.1638
Epoch 2/60
800/800 [==============================] - 9s 11ms/step - loss: 4.1310 - acc: 0.1787
Epoch 3/60
800/800 [==============================] - 9s 11ms/step - loss: 3.7652 - acc: 0.2675
Epoch 4/60
800/800 [==============================] - 9s 11ms/step - loss: 3.6079 - acc: 0.3087
Epoch 5/60
800/800 [==============================] - 9s 11ms/step - loss: 3.4734 - acc: 0.3262
Epoch 6/60
800/800 [==============================] - 9s 11ms/step - loss: 3.3337 - acc: 0.3313
Epoch 7/60
800/800 [==============================] - 9s 11ms/step - loss: 3.2012 - acc: 0.3337
Epoch 8/60
800/800 [==============================] - 9s 11ms/step - loss: 3.0662 - acc: 0.3450
Epoch 9/60
800/800 [==============================] - 9s 11ms/step - loss: 2.9428 - acc: 0.3500
Epoch 10/60
800/800 [==============================] - 9s 11ms/step - loss: 2.8100 - acc: 0.3650
Epoch 11/60
800/800 [=================

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [47]:
y_test = np.asarray(y_enc[800:])


y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))


                                                                                                                              precision    recall  f1-score   support

4 - In Review|Area-Compilers|Area-Language Design|Bug|Language-C#|New Language Feature - Default Interface Impl|Urgency-Soon       0.00      0.00      0.00         1
                                            4 - In Review|Area-Compilers|Bug|New Language Feature - Nullable Reference Types       0.00      0.00      0.00         1
                                                    4 - In Review|Area-Compilers|Bug|New Language Feature - Pattern Matching       0.00      0.00      0.00         0
                                                                                   Approved to merge|Area-Compilers|ask-mode       0.00      0.00      0.00         1
                                                                                                  Approved to merge|Area-IDE       0.00      0.00      0.00         1
   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section